In [1]:
import os

In [2]:
%pwd

'c:\\Users\\cpatil\\Desktop\\cpatil\\NLP_Text_Summarizer_Project-main\\research'

In [3]:
# to perform everything from root folder
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\cpatil\\Desktop\\cpatil\\NLP_Text_Summarizer_Project-main'

# WORKFLOW steps:

# 1. update config.yaml

# 2. SKIPPING update params.yaml . We will update it after setting model trainer configurations

# 3. update entity (return datatype of a function) in 01_data_ingestion.ipynb

In [5]:
# entity
# to define return datatype of a function import libraries.
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path   # since in configurations (config.yaml) whatever inputs we have set, we need to specify the type of the input here.
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# WORKFLOW steps:

# 4. Update the configuration manager in src config. But we are doing it here. see below.
This is is done in order to save path of config.yaml as a constant and call it whenever and wherever we want.


# go to update constants--> __init__.py for importing it below.

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # ConfigBox allows calling dict.key as here we have called config.artifacts_root mentioned in config.yaml.

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir]) 

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

# WORKFLOW steps:

# 5. Update the components.

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    # to download the data from URL
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    # to extract the downloaded zip file.
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

# WORKFLOW steps:

# 6. Update the pipeline.

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-16 21:03:27,265: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-16 21:03:27,268: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-16 21:03:27,270: INFO: common: created directory at: artifacts]
[2023-07-16 21:03:27,272: INFO: common: created directory at: artifacts/data_ingestion]


[2023-07-16 21:03:28,411: INFO: 2940924963: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4098:F66F3:83C7A:D20F3:64B40312
Accept-Ranges: bytes
Date: Sun, 16 Jul 2023 15:33:27 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100023-HYD
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1689521607.296917,VS0,VE285
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 6d97e42ac42153361634d250846223b4ed00f7f3
Expires: Sun, 16 Jul 2023 15:38:27 GMT
Source-Age: 0

]
